In [1]:
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import os

## 데이터 불러오기

In [6]:
feature_list = ['Plate Temp', 'Steam Temp', 'Vessel Pressure', 'Vessel Temp', 'P', 'I', 'Total Time', 'ADJ Time']
p_Gain = 0.2
i_Gain = 30

In [37]:
# 기존 데이터
file_dir = './Valid Data/IC 별 데이터 분리' # csv위치 폴더
file_list = os.listdir(file_dir)

for ic in range(1,6+1):
    feature_list = ['Plate Temp', 'Steam Temp', 'Vessel Pressure', 'Vessel Temp', 'P', 'I', 'Total Time', 'ADJ Time']
    file_list = os.listdir(file_dir+'/'+f'IC{ic}')
    tmp_arr = np.zeros([len(file_list),8])
    for i,f_name in enumerate(file_list):
        tmp = pd.read_csv(file_dir + f'/IC{ic}/' + file_list[i])                           # 파일 불러오기
        tmp_arr[i, :4]   = tmp.iloc[0, 1:5]                                            # 초기값(모니터링 변수) 저장
        tmp_arr[i, 4:-2] = [p_Gain,i_Gain]                                                       # P, I값 저장
        tmp_arr[i, -2] = tmp.shape[0]                                                  # 전체 공정시간 저장 (아마 사용 X)
        idx = np.where(tmp.iloc[:, 0] == 'eSTERILIZATION')[0][0]                       # 멸균 직전 시간까지 index 가져오기
        tmp_arr[i, -1] = tmp.iloc[:idx, :].shape[0]    
        exec(f'Data{ic} = pd.DataFrame(data = tmp_arr,columns = feature_list)')

In [34]:
for ic in range(1,6+1):
    exec(f'tmp = Data{ic}')
    mean = np.mean(tmp['ADJ Time'])
    print(f'Adj Time avg of IC{ic}: {mean}') # 온도 높아질수록, 가변 시간이 짧아짐

Adj Time avg of IC1: 1802.0
Adj Time avg of IC2: 1812.2307692307693
Adj Time avg of IC3: 1818.0
Adj Time avg of IC4: 1784.2
Adj Time avg of IC5: 1696.8947368421052
Adj Time avg of IC6: 1684.0


In [76]:
# Valid 데이터
file_dir = './Valid Data/Validation Data' 
file_list = os.listdir(file_dir)
file_list.sort(key = lambda x: int(x.split('case')[-1].split('-')[0]))       # 번호순 정렬

index = []


p_Gain = 4
i_Gain = 10

tmp_arr = np.zeros([len(file_list),8])
for i,f_name in enumerate(file_list):
    tmp = pd.read_csv(file_dir +'/' + f_name)                                      # 파일 불러오기
    tmp_arr[i, :4]   = tmp.iloc[0, 1:5]                                            # 초기값(모니터링 변수) 저장
    tmp_arr[i, 4:-2] = [p_Gain,i_Gain]                                             # P, I값 저장
    tmp_arr[i, -2] = tmp.shape[0]                                                  # 전체 공정시간 저장 (아마 사용 X)
    idx = np.where(tmp.iloc[:, 0] == 'eSTERILIZATION')[0][0]                       # 멸균 직전 시간까지 index 가져오기
    tmp_arr[i, -1] = tmp.iloc[:idx, :].shape[0]
    index.append(f_name.split('case')[-1].split('.csv')[0])


v_Data = pd.DataFrame(data = tmp_arr,columns = feature_list,index=index)

# PI값 재 기입
v_Data.loc['7-2']['I'] = 14

v_Data.loc['8-1']['P'] = 2.4
v_Data.loc['8-1']['I'] = 11

v_Data.loc['8-2']['P'] = 2.4
v_Data.loc['8-2']['I'] = 12

v_Data.loc['9-1']['I'] = 14

In [78]:
for ic in range(1,6+1):
    exec(f'tmp = Data{ic}')
    mean = np.mean(tmp['ADJ Time'])
    print(f'Adj Time avg of IC{ic}: {mean}') # 온도 높아질수록, 가변 시간이 짧아짐

Adj Time avg of IC1: 1802.0
Adj Time avg of IC2: 1812.2307692307693
Adj Time avg of IC3: 1818.0
Adj Time avg of IC4: 1784.2
Adj Time avg of IC5: 1696.8947368421052
Adj Time avg of IC6: 1684.0


In [80]:
v_Data

,Plate Temp,Steam Temp,Vessel Pressure,Vessel Temp,P,I,Total Time,ADJ Time
6-1,17.3,17.7,-0.01,18.4,4.0,10.0,3367.0,1605.0
6-2,17.8,18.4,-0.01,19.4,4.0,10.0,3363.0,1604.0
7-1,21.2,21.2,-0.01,21.3,4.0,10.0,3501.0,1736.0
7-2,25.5,27.3,-0.01,26.7,4.0,14.0,3369.0,1602.0
8-1,33.2,44.2,-0.01,36.5,2.4,11.0,3389.0,1624.0
8-2,34.1,46.3,0.00,37.5,2.4,12.0,3335.0,1572.0
9-1,40.2,56.3,0.01,44.2,4.0,14.0,3290.0,1528.0
9-2,41.6,60.8,0.00,45.6,4.0,10.0,3293.0,1530.0
10-1,51.2,76.1,-0.01,53.3,4.0,10.0,3290.0,1526.0
10-2,52.4,78.7,0.01,53.7,4.0,10.0,3267.0,1505.0
